In [1]:
import numpy as np
import torch
%matplotlib inline
from matplotlib import pyplot as plt
import tqdm
import torch.nn.functional as F
import torch.nn as nn
import pickle
import fb_utils as fb
from torch.utils.data import DataLoader
import random
import pandas
import os

from model import TDFilterbank, Classifier
from dataset import TinySol
from losses import KappaLoss

In [2]:
# set seed
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(0)
np.random.seed(0)

In [3]:
config = {
        "N": 2**12,
        "J": 96,
        "T": 1024,
        "sr": 16000,
        "fmin": 64,
        "fmax": 8000,
        "stride": 512,
        "batch_size": 64,
        "epochs": 100,
    }

info_csv_path="/Users/felixperfler/Documents/ISF/Random Filterbanks/TinySOL_metadata.csv"
data_dir="/Users/felixperfler/Documents/ISF/Random Filterbanks/TinySOL2020"
# info_csv_path="/Users/Dane/Documents/GitHub/Random-Filterbanks/TinySOL_metadata.csv"
# data_dir="/Users/Dane/Documents/GitHub/Random-Filterbanks/TinySOL2020"
tiny_sol_csv = '/Users/felixperfler/Documents/ISF/Random Filterbanks/TinySOL_metadata.csv'


TinySol_df = pandas.read_csv(tiny_sol_csv)

instrument_classes = TinySol_df['Instrument (abbr.)'].unique()
# map instrument classes to integers
instrument_to_int = {instrument: i for i, instrument in enumerate(instrument_classes)}
int_to_instrument = {i: instrument for i, instrument in enumerate(instrument_classes)}


random_filterbank = fb.random_filterbank(config["N"], config["J"], config["T"], tight=False, support_only=False)
random_filterbank_tight = fb.fir_tight(random_filterbank.numpy(), config["T"], eps=1.01)
target = 'VQT'
# get current working directory of file
cwd = os.path.abspath('')

with open(cwd+'/targets/'+target+'.pkl', 'rb') as fp:
    target_filterbank = pickle.load(fp)["freqz"]
    target_filterbank = torch.from_numpy(target_filterbank.T)


In [4]:
classifier = Classifier(num_classes=len(instrument_classes))

loss = nn.CrossEntropyLoss()

train_dataset = TinySol(info_csv_path, data_dir, config, target_filterbank, 'train')
train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=4)

val_dataset = TinySol(info_csv_path, data_dir, config, target_filterbank, 'val')
val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=4)

optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)

In [6]:
for epoch in range(config["epochs"]):

    for batch in train_loader:
        x = batch['x_out']
        instrument_target = batch['instrument_vec']

        optimizer.zero_grad()
        print(x.shape)
        instrument_pred = classifier(x)
        l = loss(instrument_pred, instrument_target)
        l.backward()
        optimizer.step()

        print(l.item())

    for batch in val_loader:
        x = batch['x_out']
        instrument_target = batch['instrument_vec']

        instrument_pred = classifier(x)
        l = loss(instrument_pred, instrument_target)
        instrument = torch.argmax(instrument_pred, dim=1)
        print(int_to_instrument[instrument[0].item()])

        print(l.item())
        

        

torch.Size([64, 96, 8])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (6144x8 and 96x128)